In [1]:
#Take a look, a docsim https://radimrehurek.com/gensim/similarities/docsim.html

In [2]:
import json
import importlib
import topicvisexplorer
import pandas as pd
import warnings
from scipy import spatial
import random
from random import sample
import numpy as np
warnings.filterwarnings("ignore", category=DeprecationWarning)
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from gensim.models import LdaMulticore

import unidecode
import _prepare
import _prepare_single_topic
import gensim_helpers
import spacy
import re, numpy as np, pandas as pd

#libraries to tokenize text
from string import punctuation
from string import digits
from nltk.tokenize import TweetTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import time


%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['usernameremoved', 'link', 'remove', 'usernameremove', 'amp', 'linkremoved',' <link removed>','usernameremoved','<usernameremoved>','<linkremoved>','usernameremoved_usernameremoved','linkremoved_linkremoved'])

#this should be easy to change for users
punctuation+="¡¿<>'`"
punctuation+='"'
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])    

#Remove digits and puntuaction
remove_digits = str.maketrans(digits, ' '*len(digits))#remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans(punctuation, ' '*len(punctuation))#remove_punctuation = str.maketrans('', '', punctuation)
remove_hashtags_caracter = str.maketrans('#', ' '*len('#'))
#las palabras de los hashtag se mantiene, pero no el simbolo. 

tknzr = TweetTokenizer()

In [3]:
importlib.reload(topicvisexplorer)
vis = topicvisexplorer.TopicVisExplorer("name")
vis.load_single_corpus_data("models_output/single_corpus_europe_cambridge_analytica_lda_mallet_with_word_embedding_final_dataset_user_study_topics_labeled.pkl")


Data loaded sucessfully


In [48]:
single_corpus_data = topicvisexplorer.single_corpus_data


In [49]:
#Import word embedding
wordembedding = single_corpus_data['word_embedding_model']

In [50]:
# Get document relevant seed words
f = open('json_file_topic_splitting_test.json',) 
data = json.load(f) 
new_document_seeds_TopicA = data['new_document_seeds']['TopicA']
new_document_seeds_TopicB = data['new_document_seeds']['TopicB']
topic_id = data['topic_id'] # parte del 1
f.close()


In [51]:
pd.DataFrame(new_document_seeds_TopicA)

,0,1,2,3,4,5,6,7,8,9,10,texto_completo
0,0.068871,0.095994,0.272844,0.072363,0.068871,0.068917,0.068871,0.076613,0.068871,0.068871,0.068916,<usernameremoved> Facebook has moved more than...
1,0.071023,0.071023,0.271942,0.073198,0.086648,0.071023,0.071032,0.071025,0.071023,0.071023,0.071043,Facebook has moved more than 1.5 billion users...
2,0.069930,0.069930,0.264281,0.087818,0.069930,0.069930,0.069930,0.070139,0.072881,0.084944,0.070287,"Facebook, keen to reduce its exposure to #GDPR..."
3,0.069930,0.069930,0.264279,0.087818,0.069930,0.069930,0.069930,0.070139,0.072881,0.084944,0.070289,"Facebook, keen to reduce its exposure to #GDPR..."
4,0.067843,0.082768,0.261861,0.067843,0.067843,0.067843,0.067843,0.082774,0.082768,0.082768,0.067848,DATA PROTECTION\nFacebook hides 1.5 billion us...
5,0.074516,0.074516,0.254836,0.074516,0.074516,0.074516,0.074516,0.074521,0.074516,0.074516,0.074517,Facebook moves 1.5bn users out of reach of new...
6,0.100633,0.069930,0.254538,0.069996,0.069930,0.069930,0.069930,0.069933,0.070132,0.085113,0.069935,Before GDPR comes into force in Europe on 25 M...
7,0.074516,0.074516,0.252266,0.077067,0.074516,0.074516,0.074516,0.074519,0.074516,0.074516,0.074540,Facebook has moved more than 1.5 billion users...
8,0.074516,0.074516,0.252265,0.077067,0.074516,0.074516,0.074516,0.074519,0.074516,0.074516,0.074540,Facebook has moved more than 1.5 billion users...
9,0.072166,0.072150,0.251663,0.074200,0.072150,0.072150,0.072150,0.081046,0.072150,0.088023,0.072152,Facebook moves 1.5bn users out of reach of new...


In [52]:
pd.DataFrame(new_document_seeds_TopicB)

,0,1,2,3,4,5,6,7,8,9,10,texto_completo
0,0.080542,0.065876,0.247721,0.068022,0.082603,0.066590,0.094557,0.078375,0.065881,0.083956,0.065876,<usernameremoved> <usernameremoved> <usernamer...
1,0.081722,0.066870,0.247236,0.112519,0.066856,0.067604,0.081404,0.067236,0.066850,0.074860,0.066845,Facebook Gave Device Makers Deep Access to Dat...
2,0.079393,0.064935,0.245167,0.086254,0.064935,0.094460,0.079108,0.088035,0.064935,0.067842,0.064935,Facebook Gave Device Makers Deep Access to Dat...
3,0.079393,0.064935,0.245167,0.086254,0.064935,0.094460,0.079108,0.088035,0.064935,0.067842,0.064935,Facebook Gave Device Makers Deep Access to Dat...
4,0.091099,0.063131,0.231960,0.067111,0.090895,0.077672,0.080285,0.063923,0.104494,0.066044,0.063386,Facebook Gave Device Makers Deep Access to Dat...
5,0.082950,0.067843,0.234419,0.085514,0.072944,0.098771,0.082667,0.068119,0.067843,0.071090,0.067843,What a shock... Facebook gave device makers (s...
6,0.086648,0.071039,0.229764,0.072016,0.076618,0.071049,0.071023,0.093145,0.071028,0.086648,0.071023,"45,000 Irish Facebook users may have had their..."
7,0.074516,0.074516,0.229112,0.083201,0.090909,0.074516,0.074591,0.075062,0.074530,0.074516,0.074531,*Information Post*\n\nOn the 25th May 2018 the...
8,0.069930,0.069964,0.226930,0.091477,0.075433,0.069930,0.085018,0.070758,0.085315,0.085315,0.069930,Feds expand Facebook data sharing probe as it ...


### Clean text

In [53]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


#this should be easy to change for users
punctuation+="¡¿<>'`"
punctuation+='"'
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])    

#Remove digits and puntuaction
remove_digits = str.maketrans(digits, ' '*len(digits))#remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans(punctuation, ' '*len(punctuation))#remove_punctuation = str.maketrans('', '', punctuation)
remove_hashtags_caracter = str.maketrans('#', ' '*len('#'))
#las palabras de los hashtag se mantiene, pero no el simbolo. 

tknzr = TweetTokenizer()
def sent_to_words(sentence):
    return tknzr.tokenize(sentence)
    
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    doc = nlp(" ".join(texts)) 
    texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
        
def text_cleaner(tweet):

    tweet = tweet.translate(remove_digits)
    #tweet = tweet.lower() it wasn't a good idea,, we lost a lot of
    tweet = tweet.translate(remove_punctuation)
    tweet = tweet.translate(remove_hashtags_caracter)
    tweet = tweet.lower()

    tweet = unidecode.unidecode(tweet)
    tweet = sent_to_words(tweet)

    tweet = remove_stopwords(tweet)

    new_tweet  = []
    for elem in tweet:
        if len(elem)>0:
            new_tweet.append(elem[0])
    tweet = lemmatization(new_tweet, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    return tweet[0]


### Get data

In [54]:
name_column_text = 'texto_completo'
name_tokenizacion = 'text_cleaner'

In [55]:
PreparedData_dict_with_more_info = single_corpus_data['tinfo_collection']
list_terms_relevance = PreparedData_dict_with_more_info.loc[PreparedData_dict_with_more_info['Category'] == 'Topic'+str(topic_id)].sort_values(by='relevance', ascending=False)['Term'].tolist()
list_relevant_documents = random.sample(single_corpus_data['relevantDocumentsDict'],2000)
current_number_topics = single_corpus_data['lda_model'].num_topics

In [56]:
list_relevant_documents = pd.DataFrame(list_relevant_documents).sort_values(int(topic_id)-1, ascending=False).reset_index()



In [57]:
list_relevant_documents

,index,0,1,2,3,4,5,6,7,8,9,10,texto_completo
0,825,0.114021,0.071023,0.202677,0.073725,0.085821,0.072011,0.071023,0.077742,0.071029,0.089253,0.071675,Title: Facebook: Cambridge Analytica data had ...
1,861,0.072150,0.072292,0.198150,0.087960,0.072167,0.104190,0.072150,0.088027,0.088023,0.072196,0.072694,Legal risk limitation la #Facebook:\nIn pract...
2,612,0.077042,0.077042,0.195484,0.077069,0.077067,0.094093,0.077042,0.093710,0.077042,0.077042,0.077370,Hello there!\n\nYou may have heard about the n...
3,1533,0.075758,0.075758,0.192418,0.075758,0.092208,0.075760,0.075976,0.075761,0.092420,0.092424,0.075760,Facebook moves 1.5bn users out of reach of new...
4,319,0.086374,0.085663,0.190579,0.069930,0.069930,0.070461,0.069930,0.073170,0.084374,0.085315,0.114274,<usernameremoved> <usernameremoved> But is tha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,849,0.065876,0.066385,0.065876,0.091156,0.152355,0.108801,0.080375,0.067189,0.066479,0.131414,0.104093,Social media is a versatile and often free way...
1996,1157,0.066040,0.075677,0.065876,0.086490,0.094706,0.102484,0.117718,0.080347,0.105829,0.065876,0.138957,In an #ICO your on line community is vital in ...
1997,387,0.088137,0.089575,0.065309,0.080415,0.076477,0.111071,0.084983,0.160850,0.064020,0.066684,0.112479,"Beyond justified applauds, and the best abilit..."
1998,164,0.064935,0.070933,0.064948,0.064935,0.113226,0.239339,0.079221,0.084380,0.066968,0.079208,0.071907,<usernameremoved> Any Chance I Could Get Lates...


In [58]:
#the idea is do this only ONCE! and tenerlo precalculado para el user study
list_relevant_documents[name_tokenizacion] = list_relevant_documents[name_column_text].apply(lambda x: text_cleaner(x))


In [59]:
'''
import pickle
with open('list_relevant_documents_topic_splitting_with_text_cleaner.pickle', 'wb') as handle:
    pickle.dump(list_relevant_documents, handle, protocol=4)
'''

"\nimport pickle\nwith open('list_relevant_documents_topic_splitting_with_text_cleaner.pickle', 'wb') as handle:\n    pickle.dump(list_relevant_documents, handle, protocol=4)\n"

In [60]:
#GUARDAR ESTO COMO UN PICKLE, ES PARA ACELAR EL TOPIC SPLITTING CON EL TEXT_CLEANER!!!

#21:35. FALTA TENER PRECALCULADO LOS DOCUMENT VECTORES!!
    
    

In [61]:
list_relevant_documents = list_relevant_documents.to_dict('records')

In [62]:
#guardarlo como pickle

In [63]:
new_document_seeds_TopicA = pd.DataFrame(new_document_seeds_TopicA).reset_index()
new_document_seeds_TopicA[name_tokenizacion] = new_document_seeds_TopicA[name_column_text].apply(lambda x: text_cleaner(x))
new_document_seeds_TopicA = new_document_seeds_TopicA.to_dict('records')

In [64]:
new_document_seeds_TopicB = pd.DataFrame(new_document_seeds_TopicB).reset_index()
new_document_seeds_TopicB[name_tokenizacion] = new_document_seeds_TopicB[name_column_text].apply(lambda x: text_cleaner(x))
new_document_seeds_TopicB = new_document_seeds_TopicB.to_dict('records')

### 1. Calculate vector documents given by the users

In [65]:
#ojo,q uizas la lista de relevancia es demasiadaaaa larga! esta todo el vocabulario creo...

In [66]:
def getDocumentVector(text, wordembedding ,  list_terms_relevance):   

    document_vector = np.array([0.0]*300)# 300 dimensions
    
    for word in text:  
        #print('esta es una word', word)
        if word in list_terms_relevance:
            raking_word = float(list_terms_relevance.index(word)+1)
            if word in wordembedding: #if word in wordembedding.wv:
                #print("WORD FOUND", word, raking_word)
                document_vector+=wordembedding[word]/raking_word #aqui hay que ponderar
            else:
                pass
                #print("WARNING, Word not found:", word)    

    return document_vector
            

In [67]:
#var = "#NYCACC #DogsOfTwitter ROCKY #ATTENTION EVERY1 WHOS INTERESTED IN #FOSTERING (for no costs to u) OR #ADOPTING A #DOG ON #DEATHROW NOT BEING W #FACEBOOK MAIL helpdogs<usernameremoved> OR MustLoveDogsNYC<usernameremoved> THEY CONTACT #RESCUES &amp; #GUIDE U <link removed>"

In [68]:
#getDocumentVector(var, wordembedding ,  list_terms_relevance)





In [69]:
#----->ESTO PODRIA HACER UN APPLY. CADA FOR ES UN APPLY. 
#----..buscar una libreria para hacer tutoriales. 


In [70]:
def get_initial_document_vector_by_class(list_terms_relevance, topic_id, name_tokenizacion,documents_class_A, documents_class_B, wordembedding):
    relevantDocumentsvector_class_A = 0.0
    list_documents_A = []
    for row in documents_class_A:
        current_text = row[name_tokenizacion]
        current_contribution = row[str(int(topic_id) -1)]
        current_document_vector = getDocumentVector(current_text, wordembedding,  list_terms_relevance)    
        relevantDocumentsvector_class_A+= current_contribution*current_document_vector
        list_documents_A.append((current_contribution, current_text))
    
    relevantDocumentsvector_class_B= 0.0
    list_documents_B = []
    for row in documents_class_B:
        current_text = row[name_tokenizacion]
        current_contribution = row[str(int(topic_id) -1)]
        current_document_vector = getDocumentVector(current_text, wordembedding,  list_terms_relevance)    
        relevantDocumentsvector_class_B+= current_contribution*current_document_vector
        list_documents_B.append((current_contribution, current_text))

    return(relevantDocumentsvector_class_A, relevantDocumentsvector_class_B, list_documents_A, list_documents_B)


In [71]:

def create_two_list_of_documents(list_relevant_documents, topic_id, name_tokenizacion,name_column_text, new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding):
    vector_A, vector_B, seeds_documents_A, seeds_documents_B = get_initial_document_vector_by_class(list_terms_relevance, topic_id, name_tokenizacion,new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding)

    vector_A = vector_A.reshape(-1, 1)
    vector_B = vector_B.reshape(-1, 1)
    documents_A = []
    documents_B = []
    most_relevant_documents_topic = set()

    list_relevant_documents = pd.DataFrame(list_relevant_documents).sort_values(int(topic_id)-1, ascending=False).reset_index()
    #print('vectores crreadoos', vector_A, vector_B)
    for index, row in list_relevant_documents.head(20000).iterrows():
        if index%100 == 0:
            print(index)
        current_contribution = row[int(topic_id)-1]
        current_text = row[name_tokenizacion]
        current_document_vector = getDocumentVector(current_text, wordembedding, list_terms_relevance).reshape(-1, 1)
        #similarity_vectorA_currentvector =  1 - spatial.distance.cosine(vector_A, current_document_vector)
        #similarity_vectorB_currentvector =  1 - spatial.distance.cosine(vector_B, current_document_vector)
        similarity_vectorA_currentvector = np.arccos(spatial.distance.cosine(vector_A, current_document_vector)-1) / np.pi
        similarity_vectorB_currentvector = np.arccos(spatial.distance.cosine(vector_B, current_document_vector)-1) / np.pi
        
        #I need this information to get the matrix of most relevant documents according to the similarity score
        
        #most_relevant_documents_topic.add((similarity_vectorA_currentvector,similarity_vectorB_currentvector,  current_contribution, row[name_column_text]))
            
        if similarity_vectorA_currentvector>= similarity_vectorB_currentvector:
            #append element to documentsA
            documents_A.append((current_contribution, row[name_tokenizacion]))
            most_relevant_documents_topic.add((similarity_vectorA_currentvector,0,  current_contribution, row[name_column_text]))

        else:
            documents_B.append((current_contribution, row[name_tokenizacion]))
            most_relevant_documents_topic.add((0,similarity_vectorB_currentvector,  current_contribution, row[name_column_text])) #QUIZAS LO CORRECTO Es que en vez de 0, sea 1-similarity_vectorB_currentvector


    print('Documentos en A', len(documents_A))
    print('Documentos en B', len(documents_B))

    #In the set of most relevant documents, we also need to calculate the similarity between the topic vector and the documents as seeds
    for row in new_document_seeds_TopicA:
        current_contribution = row[str(int(topic_id)-1)]
        current_text = row[name_tokenizacion]
        #print('agregando este texto', current_text)
        current_document_vector = getDocumentVector(current_text, wordembedding, list_terms_relevance).reshape(-1, 1)
        #angular distance: https://www.wikiwand.com/en/Cosine_similarity
        #https://math.stackexchange.com/questions/3241174/scale-cosine-similarity-between-vectors-to-range-0-1

        similarity_vectorA_currentvector = np.arccos(spatial.distance.cosine(vector_A, current_document_vector)-1) / np.pi
        #similarity_vectorB_currentvector = np.arccos(spatial.distance.cosine(vector_B, current_document_vector)-1) / np.pi
        #similarity_vectorA_currentvector =  1 - spatial.distance.cosine(vector_A, current_document_vector)
        #similarity_vectorB_currentvector =  1 - spatial.distance.cosine(vector_B, current_document_vector)
        
        most_relevant_documents_topic.add((similarity_vectorA_currentvector, 0, current_contribution, row[name_column_text]))
    
    for row in new_document_seeds_TopicB:
        current_contribution = row[str(int(topic_id)-1)]
        current_text = row[name_tokenizacion]
        #print('agregando este texto', current_text)
        current_document_vector = getDocumentVector(current_text, wordembedding, list_terms_relevance).reshape(-1, 1)
        #similarity_vectorA_currentvector =  1 - spatial.distance.cosine(vector_A, current_document_vector)
        #similarity_vectorB_currentvector =  1 - spatial.distance.cosine(vector_B, current_document_vector)
        #similarity_vectorA_currentvector = np.arccos(spatial.distance.cosine(vector_A, current_document_vector)-1) / np.pi
        similarity_vectorB_currentvector = np.arccos(spatial.distance.cosine(vector_B, current_document_vector)-1) / np.pi
        
        most_relevant_documents_topic.add((0, similarity_vectorB_currentvector, current_contribution, row[name_column_text]))
    
                
    
    return (seeds_documents_A, seeds_documents_B, documents_A, documents_B, most_relevant_documents_topic)

        
        


In [72]:
#Use gensim muticore
#Quizas sea buena idea usar mallet para mejorar el resultado

In [73]:

        
# !python3 -m spacy download en  # run in terminal once
'''
def process_words(texts):
    texts_out = [text_cleaner(doc) for doc in texts]    
    return texts_out
''';



In [74]:
def getLdaModelfromSentences(sentences):
    print('lda started')
    #data_words = list(sent_to_words(sentences))
    #data_ready = process_words(sentences)  # processed Text Data!
    # Create Dictionary
    id2word = corpora.Dictionary(sentences)

    # Create Corpus: Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in sentences]
    # Build LDA model
    print('building model')
    lda_model = LdaMulticore(corpus=corpus,
                            id2word=id2word,
                            num_topics=1, 
                            random_state=100,
                            chunksize=10,
                            passes=10,
                            alpha='symmetric',
                            iterations=100,
                            per_word_topics=True)
    print('termine de calcular el modelo')
    return (lda_model, corpus, id2word)


In [75]:
def getCorpusDictionaryfromSentences(sentences):
    print('lda started')
    #data_words = list(sent_to_words(sentences))
    #data_ready = process_words(sentences)  # processed Text Data!
    # Create Dictionary
    id2word = corpora.Dictionary(sentences)

    # Create Corpus: Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in sentences]
    # Build LDA model
    print('sendind corpus and dictionary')
    return (corpus, id2word)


In [76]:
def get_new_subtopics(list_relevant_documents, topic_id, name_tokenizacion,name_column_text, new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding):
    final_list_A = []
    final_list_B = []

    seeds_documents_A, seeds_documents_B, documents_A, documents_B, most_relevant_documents_topic = create_two_list_of_documents(list_relevant_documents, topic_id, name_tokenizacion,name_column_text, new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding)
    print('seeds documents a', len(seeds_documents_A))
    print('seeds documents b', len(seeds_documents_B))
    print('seeds documents a', len(documents_A))
    print('seeds documents a', len(documents_B))
    
    for contribution, text in seeds_documents_A:
        final_list_A.extend([text]*int(contribution*100))
    for contribution, text in seeds_documents_B:
        final_list_B.extend([text]*int(contribution*100))

    for contribution, text in documents_A:
        final_list_A.extend([text]*int(contribution*100))
    for contribution, text in documents_B:
        final_list_B.extend([text]*int(contribution*100))
    
    freq_topic_A = len(final_list_A)/(len(final_list_A)+len(final_list_B))
    freq_topic_B = len(final_list_B)/(len(final_list_A)+len(final_list_B))
    
    return(getCorpusDictionaryfromSentences(final_list_A), getCorpusDictionaryfromSentences(final_list_B), most_relevant_documents_topic, freq_topic_A, freq_topic_B)


In [77]:
import time

In [78]:
start = time.time()

model_topic_A, model_topic_B, most_relevant_documents_topic, freq_topic_A, freq_topic_B = get_new_subtopics(list_relevant_documents, topic_id, name_tokenizacion,name_column_text, new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding)

end = time.time()
print("Tiempo en realizar el topic splitting - Final Sending data", end - start)

0
100
200
300
400


c:\users\gonza\tesisenv\lib\site-packages\scipy\spatial\distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
Documentos en A 917
Documentos en B 1083
seeds documents a 17
seeds documents b 9
seeds documents a 917
seeds documents a 1083
lda started
sendind corpus and dictionary
lda started
sendind corpus and dictionary
Tiempo en realizar el topic splitting - Final Sending data 13.784395933151245


In [79]:
#chequear topic distribution 
most_relevant_documents_topic

{(0.6277983114896302,
  0,
  0.08264462809917675,
  "<link removed> If you use Facebook you agree to it. It's as simple as that tbh. People need to stop complaining."),
 (0.4855147078192379,
  0,
  0.08264462809918073,
  'THIS IS A VILE RACIST ARTICLE WRITTEN BY A WOMAN OF COLOUR: <link removed>'),
 (0,
  0.8023869629212075,
  0.09242424242427055,
  '#Facebooks Answers debuts a slightly redesigned UI and adds the number of users who have answered each question. #SocialMedia <link removed>'),
 (0.5492818858260295,
  0,
  0.08912655971479529,
  'Wow <usernameremoved> event in #aalsmeer #gdpr #GDPRcompliance #governance #BigData #DGCS18 <link removed>'),
 (0.8771254560029359,
  0,
  0.1318896369962875,
  'Friday talks at InnovatorsTribe!\nOur CTO <usernameremoved> will attend as one of the panelist. \nThe topic of discussion will be ethics &amp; use personal data related to Digital Marketing. #AI #bigdata #privacy #GDPR\n\nTODAY 3PM-6PM @ StartupVillage Amsterdam\n\n<link removed>'),
 (0.

###  Update frequency of corpus, log lift, etc.., manually!!!!


In [96]:
import gensim


In [80]:
def _extract_data_without_topic_model(corpus, dictionary):


    topic_model = None
    if not gensim.matutils.ismatrix(corpus):
        corpus_csc = gensim.matutils.corpus2csc(corpus, num_terms=len(dictionary))
    else:
        corpus_csc = corpus
        # Need corpus to be a streaming gensim list corpus for len and inference functions below:
        corpus = gensim.matutils.Sparse2Corpus(corpus_csc)

    vocab = list(dictionary.token2id.keys())
    
    beta = 0.01
    fnames_argsort = np.asarray(list(dictionary.token2id.values()), dtype=np.int_)
    term_freqs = corpus_csc.sum(axis=1).A.ravel()[fnames_argsort]
    term_freqs[term_freqs == 0] = beta

    assert term_freqs.shape[0] == len(dictionary),\
        'Term frequencies and dictionary have different shape {} != {}'.format(
        term_freqs.shape[0], len(dictionary))



    topic_term_dists = term_freqs/term_freqs.sum(axis=0) # esta bien esto! 
    return {'topic_term_dists': topic_term_dists, 'vocab': vocab, 'term_frequency': term_freqs}

In [88]:
len(pd.DataFrame(data_model_B))

2895

In [81]:
#get new distribution of terms, topic A
corpus_topic_A, dictionary_topic_A = model_topic_A
data_model_A = _extract_data_without_topic_model(corpus_topic_A, dictionary_topic_A)

corpus_topic_B, dictionary_topic_B = model_topic_B
data_model_B = _extract_data_without_topic_model(corpus_topic_B, dictionary_topic_B)

In [86]:
pd.DataFrame(data_model_B).sort_values(by='topic_term_dists', ascending=False).head(30)

,topic_term_dists,vocab,term_frequency
23,0.081402,remove,8312.0
5,0.036284,datum,3705.0
8,0.017285,facebook,1765.0
441,0.014161,bigdata,1446.0
18,0.011664,user,1191.0
144,0.010234,use,1045.0
88,0.007560,removed,772.0
77,0.006826,people,697.0
9,0.006620,give,676.0
0,0.006091,access,622.0


In [100]:
pd.DataFrame(data_model_B)

,topic_term_dists,vocab,term_frequency
0,0.039211,adopt,330.0
1,0.039211,attention,330.0
2,0.014853,contact,125.0
3,0.039211,cost,330.0
4,0.030181,dog,254.0
...,...,...,...
189,0.001901,view,16.0
190,0.001901,will,16.0
191,0.001901,deletefacebook,16.0
192,0.001901,disable,16.0


### Update most relevant documents

In [101]:
#single_corpus_data['data_dict']['topic_term_dists'].sum(axis=1)

In [102]:
new_dict = dict()

In [103]:
topic_id

1

In [104]:
current_number_topics

11

In [105]:
pd.DataFrame(single_corpus_data['relevantDocumentsDict'])

,0,1,2,3,4,5,6,7,8,9,10,texto_completo
0,0.067683,0.075224,0.111145,0.083138,0.087522,0.095556,0.066845,0.120091,0.133339,0.066889,0.092569,Meet our newest product: the transparent CamHa...
1,0.095597,0.098896,0.078370,0.078370,0.078370,0.112853,0.078630,0.078384,0.125669,0.079250,0.095611,<usernameremoved> This week has been the most ...
2,0.087413,0.087413,0.087413,0.087643,0.087413,0.087413,0.087413,0.106643,0.106413,0.087413,0.087413,Pulitzer prize for this lady. Investigative jo...
3,0.083831,0.092424,0.136948,0.097352,0.075758,0.109086,0.077989,0.089814,0.075758,0.076336,0.084705,"Theres no more secrecy, confident and privacy ..."
4,0.081169,0.148244,0.096213,0.093189,0.081169,0.081169,0.093545,0.081761,0.081204,0.081169,0.081169,AggregateIQ: the obscure Canadian tech firm an...
...,...,...,...,...,...,...,...,...,...,...,...,...
111740,0.081820,0.072192,0.078380,0.178688,0.072150,0.072177,0.119700,0.088103,0.072150,0.092488,0.072150,Calling all #startups that can bring value to ...
111741,0.156446,0.078370,0.080614,0.080978,0.078370,0.078383,0.078370,0.078370,0.109758,0.098949,0.081393,<usernameremoved> <usernameremoved> <usernamer...
111742,0.167842,0.079745,0.079745,0.079745,0.096877,0.079745,0.079779,0.086952,0.079745,0.080613,0.089214,Nothing makes me more upset than when Facebook...
111743,0.089127,0.089127,0.089127,0.089127,0.089127,0.089127,0.089127,0.089127,0.108734,0.089127,0.089127,Do you use hashtags in your Facebook ads? This...


In [108]:
most_relevant_documents_topic

{(-0.1392070828690095,
  -0.0037881371809806996,
  0.16738816738775206,
  "<usernameremoved> Facebook has 6 lobbyists with EP accreditations. Remove their accreditations and pressure the EC not to take any meeting with Facebook lobbyists/representatives until Mr. Zuckerberg answers concerns in front of the EP. Sometimes, it doesn't take a genius."),
 (0.1653613557674567,
  0.3373469113145058,
  0.16864108751862517,
  "Going through my Facebook ad data. Apparently my interests include 'Hair Products', 'Women's Gymnastics', 'The Bible', and 'Brazilian Thrash Metal'. so ads are 50% correct basically."),
 (0.24812272464228768,
  0.11481589294005956,
  0.17290360509404806,
  'Retweeted Ambassador Rob (<usernameremoved>\n\nTrump called Maxine Waters low IQ, no call for civility. \n\nTrump called NFL players Son-of-Bitches no call for civility. \n\nMAGA folks shouted lock Her Up no call for... <link removed>'),
 (0.26957174456779254,
  0.3696736361801627,
  0.21629191265725176,
  '.\n#NYCACC\

In [106]:
#set columns of the new subtopics to NaN values
df = pd.DataFrame(single_corpus_data['relevantDocumentsDict'])

df[int(topic_id-1)]= 0.0
df[current_number_topics]= 0.0

for row in most_relevant_documents_topic:
    contribution_to_topic_a = row[0]/(row[0]+row[1])
    contribution_to_topic_b = row[1]/(row[0]+row[1])
    #print(contribution_to_topic_a, contribution_to_topic_b)
    previous_contribution = row[2]
    indexs = df.index[df['texto_completo'] == row[-1]].tolist()
    if len(indexs)<1:
        print('Error, text not found')
    #set final contribution to topic a, is contribution to topic_a multiply by the previous contribuiton
    df.loc[indexs,int(topic_id-1)] = contribution_to_topic_a*previous_contribution
    df.loc[indexs,current_number_topics] = contribution_to_topic_b*previous_contribution
    #print(previous_contribution, contribution_to_topic_a*previous_contribution+contribution_to_topic_b*previous_contribution )
        
    
#order columns
intList=sorted([i for i in df.columns.values if type(i) is int])
strList=sorted([i for i in df.columns.values if type(i) is str])
new_order = intList+strList
df = df[new_order]


In [107]:
df[df[11]>1]

,0,1,2,3,4,5,6,7,8,9,10,11,texto_completo


In [71]:
single_corpus_data['relevantDocumentsDict'] = df.to_dict('records')
new_dict['relevantDocumentsDict_fromPython'] =json.dumps( single_corpus_data['relevantDocumentsDict'])


In [72]:
pd.DataFrame(single_corpus_data['relevantDocumentsDict'])

,0,1,2,3,4,5,6,7,8,9,10,11,texto_completo
0,0.0,0.075224,0.111145,0.083138,0.087522,0.095556,0.066845,0.120091,0.133339,0.066889,0.092569,0.0,Meet our newest product: the transparent CamHa...
1,0.0,0.098896,0.078370,0.078370,0.078370,0.112853,0.078630,0.078384,0.125669,0.079250,0.095611,0.0,<usernameremoved> This week has been the most ...
2,0.0,0.087413,0.087413,0.087643,0.087413,0.087413,0.087413,0.106643,0.106413,0.087413,0.087413,0.0,Pulitzer prize for this lady. Investigative jo...
3,0.0,0.092424,0.136948,0.097352,0.075758,0.109086,0.077989,0.089814,0.075758,0.076336,0.084705,0.0,"Theres no more secrecy, confident and privacy ..."
4,0.0,0.148244,0.096213,0.093189,0.081169,0.081169,0.093545,0.081761,0.081204,0.081169,0.081169,0.0,AggregateIQ: the obscure Canadian tech firm an...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111740,0.0,0.072192,0.078380,0.178688,0.072150,0.072177,0.119700,0.088103,0.072150,0.092488,0.072150,0.0,Calling all #startups that can bring value to ...
111741,0.0,0.078370,0.080614,0.080978,0.078370,0.078383,0.078370,0.078370,0.109758,0.098949,0.081393,0.0,<usernameremoved> <usernameremoved> <usernamer...
111742,0.0,0.079745,0.079745,0.079745,0.096877,0.079745,0.079779,0.086952,0.079745,0.080613,0.089214,0.0,Nothing makes me more upset than when Facebook...
111743,0.0,0.089127,0.089127,0.089127,0.089127,0.089127,0.089127,0.089127,0.108734,0.089127,0.089127,0.0,Do you use hashtags in your Facebook ads? This...


## Update prepared data

In [73]:
topic_id
#

1

In [74]:
temp = single_corpus_data['PreparedDataObtained']

In [75]:
temp.keys()

dict_keys(['mdsDat', 'tinfo', 'token.table', 'R', 'lambda.step', 'plot.opts', 'topic.order'])

In [76]:
#update MdsDat
#add temporal coordinates. We are going to change these later with the new topic similarity metric.
temp['mdsDat']['x'].append(temp['mdsDat']['x'][topic_id-1])
temp['mdsDat']['y'].append(temp['mdsDat']['y'][topic_id-1])
temp['mdsDat']['topics'].append(len(temp['mdsDat']['topics'])+1)
temp['mdsDat']['cluster'].append(temp['mdsDat']['cluster'][topic_id-1])
#update the frequency of the topic
old_frequency = temp['mdsDat']['Freq'][topic_id-1]
temp['mdsDat']['Freq'][topic_id-1] = old_frequency*freq_topic_A
temp['mdsDat']['Freq'].append(old_frequency*freq_topic_B)
#Update topic.order
temp['topic.order'].append(len(temp['topic.order'])+1)

In [83]:
temp_tinfo_df

,Category,Freq,Term,Total,loglift,logprob
0,Default,19023.0,datum,19023.000000,31977.0,31977.0
1,Default,10705.0,bigdata,10705.000000,31976.0,31976.0
2,Default,8660.0,privacy,8660.000000,31975.0,31975.0
3,Default,9296.0,facebook,9296.000000,31974.0,31974.0
4,Default,6147.0,user,6147.000000,31973.0,31973.0
...,...,...,...,...,...,...
383719,Topic11,0.0,riskblock,1.229759,-inf,-inf
383720,Topic11,0.0,thirdpartie,1.103186,-inf,-inf
383721,Topic11,0.0,netcommfocus,1.231039,-inf,-inf
383722,Topic11,0.0,sharmishta,1.216717,-inf,-inf


### Change frequency of terms in first new subtopic 

In [89]:
def change_frequency(row, new_subtopic_df, total_sum_frequency_corpus):
    current_term = row['Term']
    current_total = row['Total']
    if current_term in list(new_subtopic_df['vocab']) and current_total>0:
        new_subtopic_df = pd.DataFrame(new_subtopic_df)
        old_freq = row['Freq']
        new_prob = float(new_subtopic_df.loc[new_subtopic_df['vocab'] == current_term]['topic_term_dists'])

        row['Freq'] = new_prob*row['Total']
        row['logprob'] = np.log(new_prob)
        row['loglift'] = np.log(new_prob/(current_total/total_sum_frequency_corpus))                 
    else:
        row['Freq'] = 0    
        row['logprob'] = 0
        row['loglift'] = 0  
    return row
    

    

In [90]:
temp_tinfo_df = pd.DataFrame(temp[ 'tinfo'])
temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(topic_id)].sort_values(by=['Freq'], ascending=False)

,Category,Freq,Term,Total,loglift,logprob
34638,Topic1,4571.303103,people,6912.165421,1.9830,-2.8971
32179,Topic1,2889.111934,delete,2889.111934,2.3965,-3.3559
32604,Topic1,2688.361586,year,2688.361586,2.3965,-3.4279
32393,Topic1,2173.182983,account,2173.182983,2.3965,-3.6407
34358,Topic1,1799.497094,friend,1801.930529,2.3951,-3.8294
...,...,...,...,...,...,...
44984,Topic1,0.000000,explanatory,1.204334,-inf,-inf
44983,Topic1,0.000000,pill,11.115410,-inf,-inf
44982,Topic1,0.000000,antiterrorism,1.216717,-inf,-inf
44981,Topic1,0.000000,nanotechnology,3.536621,-inf,-inf


In [91]:
total_sum_frequency_corpus = sum(temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(topic_id)].sort_values(by=['Freq'], ascending=False)['Total'])

In [92]:
total_sum_frequency_corpus

909905.0000001481

In [93]:
temp_tinfo_df = pd.DataFrame(temp[ 'tinfo'])
temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(topic_id)] = temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(topic_id)].apply(lambda row: change_frequency(row, data_model_A, total_sum_frequency_corpus), axis=1)
temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(topic_id)].sort_values(by=['Freq'], ascending=False)

,Category,Freq,Term,Total,loglift,logprob
35535,Topic1,301.444110,datum,19023.874053,-0.277220,-4.144865
35595,Topic1,159.644885,facebook,9296.681524,0.519222,-4.064461
35547,Topic1,119.364693,bigdata,10705.321411,-0.053714,-4.496313
35515,Topic1,57.191784,privacy,8660.746780,-0.365607,-5.020146
32026,Topic1,48.982889,remove,483.735778,5.249489,-2.290068
...,...,...,...,...,...,...
43826,Topic1,0.000000,aion,1.197970,0.000000,0.000000
43825,Topic1,0.000000,sweetie,3.447615,0.000000,0.000000
43824,Topic1,0.000000,robustly,6.950360,0.000000,0.000000
43823,Topic1,0.000000,reputedly,1.149205,0.000000,0.000000


### Change frequency of terms in second new subtopic

In [ ]:
#copy values for the new subtopic b
temp2 = temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(topic_id)]
temp2.Category = 'Topic'+str(current_number_topics+1) 
temp_tinfo_df = temp_tinfo_df.append(temp2, ignore_index=True)


In [ ]:
#update those values with the current terms probability
temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(current_number_topics+1)] = temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(current_number_topics+1)].apply(lambda row: change_frequency(row, data_model_B), axis=1)
temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(current_number_topics+1)].sort_values(by=['Freq'], ascending=False)

In [ ]:
#save the new tinfo
temp_tinfo_df.reset_index(drop=True, inplace=True)
temp[ 'tinfo']  = temp_tinfo_df.to_dict(orient='list')

In [ ]:
single_corpus_data['PreparedDataObtained'] = temp

# Indicate new number of topics

In [ ]:
single_corpus_data['lda_model'].num_topics = single_corpus_data['lda_model'].num_topics+1
single_corpus_data['lda_model'].num_topics

# Get new topic similarity matrix

In [ ]:
newClass = topicvisexplorer.TopicVisExplorer("name") #borrar esta
#newClass = TopicVisExplorer("name") #dejar esta en el codigo final

In [ ]:

word_embedding_model = single_corpus_data['word_embedding_model']
topn_terms = 20
topk_documents = 20
relevance_lambda = 0.6
print('Calculando topic similarity metrix')

In [ ]:
lda_model = single_corpus_data['lda_model']
corpus = single_corpus_data['corpus']
id2word = single_corpus_data['id2word']


In [ ]:
matrix_documents_topic_contribution = pd.DataFrame(single_corpus_data['relevantDocumentsDict'])


In [ ]:


new_topic_similarity_matrix = newClass.calculate_topic_similarity_on_single_corpus(word_embedding_model, lda_model, corpus, id2word, matrix_documents_topic_contribution,topn_terms, topk_documents, relevance_lambda)


In [ ]:
single_corpus_data['topic_similarity_matrix'] = new_topic_similarity_matrix
print('Topic similarity matrix has been calculated')
